In [1]:
'''
Maarten Janzen, I6246698
January 2025
Master thesis internship

final version of all functions needed to generate: 
 - excel sheet for collection
 - prompts
 - images

files needed:
 - interview files (in interviews/)
 - template prompts (in templates/, saved as '[category]_template.txt')
 - excel file (called 'interview_info.xlsx')
 
functions:
 - setup
     param: participant
     desc: checks if necessary directories and files exist, creates them if not. 
           if the files for the prompt templates dont exist, they will be created empty
 - check_if_exists
     param: file to check and directory to check and save to
     desc: checks if the filename already exists in the target directory
           if so it adds a number to the end of the file (e.g. file_1).
           this number is incremented until there is no overwrite necessary
 - headers_updated
     param: sheet name (to add headers to)
     desc: adds headers to the excel file in row 1. will overwrite already existing values
           if the sheet does not exist, it is created.
 - coordinate_from_string
     param: coordinate (e.g. 'A9')
     desc: returns coordinate in number (e.g. (1, 9)). rewrite of openpyxl function 
           (did not work)
 - myMedsArentWorking
     param: template prompt (in templates folder specific for category), marker 
            marking specific part of the string that can be replaced, splitter 
            to split positive and negative prompts, item to insert into template 
            to create new prompt
     desc: badly names function that generates a single prompt and negative prompt 
           per time it is called
 - last_entry
     param: row, sheet
     desc: returns the last cell in a row where a value is present
 - progress_bar
     param: current value, total value
     desc: creates a progress bar to show progress when generating images. currently 
           functional but could be improved (starts at 2% ends at 98%)
 - sort_answers
     param: interview/answer file (.txt), output file (excel file), participant (will be sheetname)
     desc: pulls questions, valence and answers from the text file and inserst them 
           into their designated columns and rows of the excel file
           DOES NOT GENERATE PROMPT
 - generate_prompts
     param: sheetname
     desc: reads into excel file to find an item, then generates prompts based on 
           the prompt templates, templates are selected depending on category.
           templates can be found in templates/ and should be in .txt.
           does not generate prompts for questions in the 'skip' variable.
           all generated prompts are written to the excel file in columns behind the item

 - generate_images_final
     param: sheet
     desc: generates images by reading out the prompts from the excel file. images
           are saved by set (1 set = 1 item = 3 prompts). images are named according to 
           the item, prompt version and iteration: '[item] prompt_[version]_[iteration].png'
           sets are added to the excel file after saving to keep track of what images are 
           saved where. output is directed to 'batch/[participant]/'.

 '''

"\nMaarten Janzen, I6246698\nJanuary 2025\nMaster thesis internship\n\nfinal version of all functions needed to generate: \n - excel sheet for collection\n - prompts\n - images\n\nfiles needed:\n - interview files (in interviews/)\n - template prompts (in templates/, saved as '[category]_template.txt')\n - excel file (called 'interview_info.xlsx')\n \nfunctions:\n - setup\n     param: participant\n     desc: checks if necessary directories and files exist, creates them if not. \n           if the files for the prompt templates dont exist, they will be created empty\n - check_if_exists\n     param: file to check and directory to check and save to\n     desc: checks if the filename already exists in the target directory\n           if so it adds a number to the end of the file (e.g. file_1).\n           this number is incremented until there is no overwrite necessary\n - headers_updated\n     param: sheet name (to add headers to)\n     desc: adds headers to the excel file in row 1. will 

In [1]:
import requests
import base64
import os
import sys
import json
import re as regex
from openpyxl.utils.cell import get_column_letter
import openpyxl

In [2]:
def setup(participant):
    '''
    setup to make sure paths exist
    batch path = 'batch/'
    participant = 'batch/participant/'
    excel = 'interview_info.xlsx'
    '''
    #check batch
    if not os.path.exists('batch/'):
        os.makedirs('batch/', exist_ok = True)
    #check participant in batch
    if not os.path.exists(f'batch/{participant}/'):
        os.makedirs(f'batch/{participant}/', exist_ok = True)
    #check excel file
    if 'interview_info_1.xlsx' not in os.listdir():
        wb = openpyxl.Workbook()
        wb.save('interview_info.xslx')
    else:
        wb = openpyxl.load_workbook('interview_info.xlsx')
    #check prompt templates
    if not os.path.exists('templates/'):
        os.makedirs('templates/', exist_ok = True)
    categories = ['drinks', 'food', 'temperature', 'activities']
    for c in categories:
        try:
            with open(f'templates/{c}_template.txt', 'x') as f:
                f.write(f'{c} prompt template file')
        except FileExistsError:
            print(f'{c}_template.txt already exists')
        

In [3]:
def check_if_exists(file, direct):
    
    ''' 
    function to help prevent overwriting files, checks if 
    filename already exists in the directory and adds number if it does
    if a file with a number already exists it will increment said number by 1
    '''
    
    name_to_check = file[:file.index('.')]
    du = [(f[len(name_to_check):f.index('.')].replace('_', '')) if f[:len(name_to_check)] in file else 0 for f in os.listdir(direct)]
    du_filtered = [int(x) if x != "" else 0 for x in du]
    
    if not du_filtered:
        return f'{file[:file.index('.')]}.{file[file.index('.') + 1:]}'
    else:
        return f'{file[:file.index('.')]}_{max(du_filtered) + 1}.{file[file.index('.')+1:]}'

In [4]:
def headers_updated(sheet):
    """
    function to add headers to the columns in the excel file
    updated to not depend on hardcoded column letters
    instead uses col_idx to get a numerical representation
    """
    print(os.getcwd())
    excel_path = 'interview_info.xlsx'
    wb = openpyxl.load_workbook(excel_path)
    
    #if sheet not in wb.sheetnames:
    #    wb.create_sheet(sheet)

    count = 0
    p = sheet
    while p in wb.sheetnames:
        count += 1
        p = f'{sheet}_{count}'
        #print(f'double --> {participant}')
    wb.create_sheet(p)

    sh = wb[p]
    headers = ['question', 'valence', 'category', 'item', 'prompt 1', 'negative 1', 'prompt 2', 'negative 2', 'prompt 3', 'negative 3', 'sets', 'score 1', 'score 2', 'score 3', 'final set', 'description']
    
    for col in range(1, 1 + len(headers)):
        
        sh.cell(row=1, column=col).value = headers[col - 1]

    wb.save('interview_info.xlsx')
    wb.close()

    print(f'headers added to sheet {p}')
    return p

In [5]:
def coordinate_from_string(coordinate):
    #openpyxl function no worky
    #only works for cell object
    letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    x = letters.index(coordinate[0]) + 1
    y = int(coordinate[1])
    return (x, y)


In [6]:
def myMedsArentWorking(prompt, marker, splitter, item):

    """
                       (  )/  
                        )(/
     ________________  ( /)
    ()__)____________))))) 

    creates propmt or smt idk
    """
    
    sp = prompt.split(splitter)
    pos = sp[0]
    neg = sp[1]

    pos_start = pos[:pos.find(marker[0])]
    pos_end = pos[pos.find(marker[1]) + 1:]

    return [pos_start + item + pos_end, neg]

In [7]:
def last_entry(row, sheet):
    max_col = sheet.max_column
    for col in range(max_col, 0, -1):
        cell = sheet.cell(row=row, column=col)
        if cell.value is not None: return cell
    return None

In [8]:
def progress_bar(current, total, bar_length = 60):
    """
    very simple progress bar to give me something to look at
    """
    fraction = current / total
    filled = int(bar_length * fraction)
    bar = "=" * filled + '-' * (bar_length - filled)
    percentage = fraction * 100
    sys.stdout.write(f'\r ||{bar}|| {percentage: .2f}%')
    sys.stdout.flush()

In [9]:
def sort_answers(file, output_file, participant):

    '''
    function that sorts the answers and questions of the interview
    generates a sheet in the excel file under the participant name
    with the categories:
    - questions      (A)
    - valence        (B)
    - category       (C)
    - item           (D)
    - prompt p1      (E)
    - prompt n1      (F)
    - prompt p2      (G)
    - prompt n2      (H)
    - prompt p3      (I)
    - prompt n4      (J)
    - path img grid 1(K) --> sets
    does not generate prompt
    OUTPUT FILE: interview_info.xlsx
    input files in directory ./interviews/ 
    '''
    with open(f'./interviews/{file}', 'r') as q:
    
        data = q.readlines()
        d = [x.strip('\n') for x in data if x.find('\n') != 0 ] #filter out empty lines (newline at 0)

        #regex patterns
        category_pattern = regex.compile(r"\d+\.\s+[a-zA-Z& ]+$")
        question_pattern = regex.compile(r"\d+\.\s+.+[-+=]$")
        answer_pattern = regex.compile(r"^>\s*[a-zA-Z\s]+(?:-->|->)?\s*.+$")
    
        wb = openpyxl.load_workbook(output_file)

        #select participant sheet as active sheet
        #if participant not in wb.sheetnames:
        #   wb.create_sheet(participant)
        sheet = wb[participant]
        
        #add headers
        #if sheet['A1'].value != 'question':
        #    headers_updated(participant)
        index = 2 #row 1 has headers 
        #find set column
        col_vals = []
        for i in range(1, sheet.max_column + 1):
            col_vals = col_vals + [sheet.cell(row=1, column=i).value]
        
        question_col = col_vals.index('question')  + 1
        category_col = col_vals.index('category')  + 1
        valence_col = col_vals.index('valence')    + 1
        item_col = col_vals.index('item')          + 1
        set_col = col_vals.index('sets')           + 1
        
        for x in d:
            #collect category
            if regex.search(category_pattern, x.strip()) != None:
                
                current_cat = x.strip()
            #collect question and valence
            elif regex.search(question_pattern, x.strip()) != None:
                
                current_q = x.strip()
                current_valence = x.strip()[-1]
            #dump everything collected for every answer in the corresponding row and column
            elif regex.search(answer_pattern, x.strip()) != None:

                #sheet[f'A{index}'] = regex.sub(r"[-+=]", "", current_q) #add questions stripped from markers
                #sheet[f'B{index}'] = current_valence #add valence
                #sheet[f'C{index}'] = current_cat #add category
                sheet.cell(row=index, column=question_col).value = regex.sub(r"[-+=]", "", current_q)
                sheet.cell(row=index, column=valence_col).value = current_valence
                sheet.cell(row=index, column=category_col).value = current_cat
                #remove /
                ans = regex.sub(r">", "", x.strip(',/ ')) #add answers
                ans = ans.split('→')[0]

                #if broken now remove from here 
                ans_filtered = [i if i != '/' else ' ' for i in ans]
                ans_final = ''

                for a in ans_filtered:
                    ans_final += a            
                #to here
                #sheet[f'D{index}'] = ans_final #was ans
                sheet.cell(row=index, column=item_col).value = ans_final
                sheet.cell(row=index, column=set_col).value = f'set_{index - 1}'
                index += 1
                
        wb.save(output_file)
        wb.close()
        print('sorted')
        #return p

In [10]:
def generate_prompts(sheet):
    '''
    function that generates the prompts required according
    to the answers of the interview sorted into the excel 
    file. inserts prompts into excel file at the corresponding item

    2 prompts per category --> 2 * 2 * 2 = 8 images
    new prompt --> 4 images

    NEW 3 PROMPTS
    3 prompts * 2 iterations * 2 images per batch = 12 images
    '''
    
    wb = openpyxl.load_workbook('interview_info.xlsx')
    sh = wb[sheet]
    markers = '[]' #marks part of the prompt that can be cut and replaced
    #skip these questions for now --> general preference no specific food
    skip = ['5. what typically draws you to a dish? does plating matter?', 
            '6. what typically drives you away from a dish?',
            '8. how do you feel about pairing your favourite food with your favourite drink?',
            '5. what typically draws you to a drink?',
            '6. what typically drives you away from a drink?',
            '3. do you have a preference for drinking receptacles (e.g. ceramic over glass)?']

    #use headers to find prompt columns (probably shouldve been a function)
    col_vals = []
    for i in range(1, sh.max_column + 1):
        col_vals = col_vals + [sh.cell(row=1, column=i).value]

    #first_prompt_col = col_vals.index('prompt 1') + 1
    prompt_cols = [i + 1 for i, j in enumerate(col_vals) if 'prompt' in j]
    neg_cols = [i + 1 for i, j in enumerate(col_vals) if 'negative' in j]
    #print(prompt_cols)
    #print(neg_cols)
    #first_neg_col = col_vals.index('negative 1') + 1
    category_col = col_vals.index('category') + 1
    item_col = col_vals.index('item') + 1
    
    for i in range(2, sh.max_row):
        if sh[f'A{i}'].value.strip() not in skip:
            #find category and item
            #category = sh[f'C{i}'].value[2:].strip()
            category = sh.cell(row=i, column=category_col).value[2:].strip()
            if category == 'temperature': 
                break
            #item = sh[f'D{i}'].value.strip()
            item = sh.cell(row=i, column=item_col).value.strip()
            #print(f'{category}_template.txt --> {item}')

            with open(f'./templates/{category}_template.txt', 'r') as t:
                #find template
                templates = t.readlines()
                #for every prompt (seperated by ||) replace the correct segment with the corresponding item from the list and place it in the correct column
                for j, t in enumerate(templates):
                    #generate prompt
                    p = myMedsArentWorking(t, markers, '||', item)
                    #select correct column
                    #max_col = last_entry(i, sh)
                    pos = p[0].strip()
                    neg = p[1].strip()
                    #pos_col = get_column_letter(max_col.column + 1)
                    #neg_col = get_column_letter(max_col.column + 2)
                    #sh[f'{pos_col}{i}'] = pos
                    #sh[f'{neg_col}{i}'] = neg
                    #print(i)
                    sh.cell(row=i, column=prompt_cols[j]).value = pos
                    sh.cell(row=i, column=neg_cols[j]).value = neg

    wb.save('interview_info.xlsx')
    wb.close()  

In [11]:
def generate_images_final(sheet):

    '''
    prompt --> E G I
    neg --> F H J 
    all items should have a set folder
    '''

    wb = openpyxl.load_workbook('interview_info.xlsx')
    sh = wb[sheet]
    prompts, neg = [], []
    set_count = 0

    col_vals = []
    for i in range(1, sh.max_column + 1):
        col_vals = col_vals + [sh.cell(row=1, column=i).value]

    prompt_cols = [i + 1 for i, j in enumerate(col_vals) if 'prompt' in j]
    neg_cols = [i + 1 for i, j in enumerate(col_vals) if 'negative' in j]
    set_col = col_vals.index('sets') + 1
    
    for i in range(2, sh.max_row):
        progress_bar(i, sh.max_row -2)

        if sh.cell(row=i, column=prompt_cols[0]).value == None: continue

        prompts = [sh.cell(row=i, column=c).value for c in prompt_cols]
        #print(prompts)
        #for n, prompt in enumerate(prompt_cols):
        for n, prompt in enumerate(prompts):
            #print(prompt)
            
            if sh.cell(row=i, column=prompt_cols[n]).value == None: continue

            payload.update({'prompt' : sh.cell(row=i, column=prompt_cols[n]).value.strip()})
            payload.update({'negative_prompt' : sh.cell(row=i, column=neg_cols[n]).value.strip()})
            response = requests.post(url=f'{url}/sdapi/v1/txt2img', json = payload)
            r = response.json()
    
            #output files
            #wont let me put set as a name
            s = sh.cell(row=i, column=set_col).value
            s_num = s[s.index('_') + 1:]
            output_file = f'set_{s_num}_prompt_{n+1}.png'
            output_directory = f'batch/{sheet}/set_{s_num}'
        
            if not os.path.exists(output_directory):
                os.makedirs(output_directory)
                 
            if 'images' in r and len(r['images']) > 0:
                for img in r['images']:
                    #check files to prevent overrides
                    current_output_file = check_if_exists(output_file, output_directory)
                    #print(current_output_file)
                    #with open(f'{output_directory}/{cat}_{current_output_file}', 'wb') as f:
                    with open(f'{output_directory}/{current_output_file}', 'wb') as f:
                        #save image
                        f.write(base64.b64decode(img))
                        
                wb.save('interview_info.xlsx')           
            else:
                print("key does not exist")

    wb.save('interview_info.xlsx')
    wb.close()

In [17]:
url = "http://127.0.0.1:7860/"

payload = {
    "prompt": "well seasoned steaming cooked vegetables in a black stone bowl on a table from a top view, amazing quality, realistic, good proportions, 8k hyperrealistic",
    "negative_prompt": "sketch, cartoon, unrealistic, badly drawn, bad quality, poorly drawn, poor quality, bad anatomy, wrong anatomy, disconnected limbs, ugly, unrealistic", 
    "steps": 30, #30
    "seed": -1,
    "width": 750, #750
    "height": 750, #750
    "cfg_scale": 7.5,
    "sampler_name": "DPM++ 2M",
    "n_iter": 2, #2
    "batch_size": 2 #2
}

#images for 010
participant = '010'
interview_file = f'questions {participant}.txt'
excel = 'interview_info.xlsx'

In [18]:
setup(participant)

drinks_template.txt already exists
food_template.txt already exists
temperature_template.txt already exists
activities_template.txt already exists


In [14]:
participant = headers_updated(participant)
print(participant)
sort_answers(interview_file, excel, participant)
generate_prompts(participant)

/home/i6246698@unimaas.nl/Documents/code
headers added to sheet 011
011
sorted


In [19]:
generate_images_final(participant)

 ||============================================================||  100.75%

In [1]:
#https://stackoverflow.com/questions/43022843/nvidia-nvml-driver-library-version-mismatch#comment73133147_43022843